### * Задание 4
В этом задании мы будем работать с датасетом, с которым мы уже знакомы по домашнему заданию по библиотеке `Matplotlib`, это датасет `Credit Card Fraud Detection`. Для этого датасета мы будем решать задачу классификации - будем определять, какие из транзакциции по кредитной карте являются мошенническими. Данный датасет сильно несбалансирован (так как случаи мошенничества относительно редки), так что применение метрики `accuracy` не принесет пользы и не поможет выбрать лучшую модель. Мы будем вычислять `AUC`, то есть площадь под кривой `ROC`.

Импортируйте из соответствующих модулей `RandomForestClassifier`, `GridSearchCV` и `train_test_split`.

Загрузите датасет `creditcard.csv` и создайте датафрейм **df**.

С помощью метода `value_counts` с аргументом `normalize=True` убедитесь в том, что выборка несбалансирована. Используя метод `info`, проверьте, все ли столбцы содержат числовые данные и нет ли в них пропусков. Примените следующую настройку, чтобы можно было просматривать все столбцы датафрейма: `pd.options.display.max_columns = 100`. Просмотрите первые 10 строк датафрейма **df**.

Создайте датафрейм **X** из датафрейма **df**, исключив столбец `Class`.

Создайте объект `Series` под названием **y** из столбца `Class`.

Разбейте **X** и **y** на тренировочный и тестовый наборы данных при помощи функции `train_test_spli`t, используя аргументы: `test_size=0.3, random_state=100, stratify=y.`

У вас должны получиться объекты `X_train`, `X_test`, `y_train` и `y_test`. Просмотрите информацию о их форме.

In [168]:
import warnings

import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score


warnings.filterwarnings('ignore')
pd.options.display.max_columns = 100

DATASET_PATH = 'Data/creditcard.csv'
RANDOM_STATE = 42

creditcards_df = pd.read_csv(DATASET_PATH)

In [170]:
df = creditcards_df

In [171]:
 df.isna().sum()

Time      0
V1        0
V2        0
V3        0
V4        0
V5        0
V6        0
V7        0
V8        0
V9        0
V10       0
V11       0
V12       0
V13       0
V14       0
V15       0
V16       0
V17       0
V18       0
V19       0
V20       0
V21       0
V22       0
V23       0
V24       0
V25       0
V26       0
V27       0
V28       0
Amount    0
Class     0
dtype: int64

In [172]:
df.Class.value_counts(normalize=True)

0    0.998273
1    0.001727
Name: Class, dtype: float64

In [173]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 284807 entries, 0 to 284806
Data columns (total 31 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   Time    284807 non-null  float64
 1   V1      284807 non-null  float64
 2   V2      284807 non-null  float64
 3   V3      284807 non-null  float64
 4   V4      284807 non-null  float64
 5   V5      284807 non-null  float64
 6   V6      284807 non-null  float64
 7   V7      284807 non-null  float64
 8   V8      284807 non-null  float64
 9   V9      284807 non-null  float64
 10  V10     284807 non-null  float64
 11  V11     284807 non-null  float64
 12  V12     284807 non-null  float64
 13  V13     284807 non-null  float64
 14  V14     284807 non-null  float64
 15  V15     284807 non-null  float64
 16  V16     284807 non-null  float64
 17  V17     284807 non-null  float64
 18  V18     284807 non-null  float64
 19  V19     284807 non-null  float64
 20  V20     284807 non-null  float64
 21  V21     28

In [152]:
### Memory saving function credit to https://www.kaggle.com/gemartin/load-data-reduce-memory-usage
def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.        
    """
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df

In [153]:
df = reduce_mem_usage(df)

Memory usage of dataframe is 67.36 MB
Memory usage after optimization is: 17.11 MB
Decreased by 74.6%


In [154]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 284807 entries, 0 to 284806
Data columns (total 31 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   Time    284807 non-null  float32
 1   V1      284807 non-null  float16
 2   V2      284807 non-null  float16
 3   V3      284807 non-null  float16
 4   V4      284807 non-null  float16
 5   V5      284807 non-null  float16
 6   V6      284807 non-null  float16
 7   V7      284807 non-null  float16
 8   V8      284807 non-null  float16
 9   V9      284807 non-null  float16
 10  V10     284807 non-null  float16
 11  V11     284807 non-null  float16
 12  V12     284807 non-null  float16
 13  V13     284807 non-null  float16
 14  V14     284807 non-null  float16
 15  V15     284807 non-null  float16
 16  V16     284807 non-null  float16
 17  V17     284807 non-null  float16
 18  V18     284807 non-null  float16
 19  V19     284807 non-null  float16
 20  V20     284807 non-null  float16
 21  V21     28

In [155]:
df.head(10)

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359375,-0.072754,2.537109,1.377930,-0.338379,0.462402,0.239624,0.098694,0.363770,0.090820,-0.551758,-0.617676,-0.991211,-0.311279,1.467773,-0.470459,0.208008,0.025787,0.404053,0.251465,-0.018311,0.277832,-0.110474,0.066956,0.128540,-0.189087,0.133545,-0.021057,149.625000,0
1,0.0,1.191406,0.266113,0.166504,0.448242,0.060028,-0.082336,-0.078796,0.085083,-0.255371,-0.166992,1.612305,1.065430,0.489014,-0.143799,0.635742,0.463867,-0.114807,-0.183350,-0.145752,-0.069092,-0.225830,-0.638672,0.101318,-0.339844,0.167114,0.125854,-0.008980,0.014725,2.689453,0
2,1.0,-1.358398,-1.339844,1.773438,0.379883,-0.503418,1.800781,0.791504,0.247681,-1.514648,0.207642,0.624512,0.066101,0.717285,-0.165894,2.345703,-2.890625,1.110352,-0.121338,-2.261719,0.524902,0.248047,0.771484,0.909180,-0.689453,-0.327637,-0.139038,-0.055359,-0.059753,378.750000,0
3,1.0,-0.966309,-0.185181,1.792969,-0.863281,-0.010307,1.247070,0.237549,0.377441,-1.386719,-0.054962,-0.226440,0.178223,0.507812,-0.287842,-0.631348,-1.059570,-0.684082,1.965820,-1.232422,-0.208008,-0.108276,0.005272,-0.190308,-1.175781,0.647461,-0.221924,0.062744,0.061462,123.500000,0
4,2.0,-1.158203,0.877930,1.548828,0.403076,-0.407227,0.095947,0.592773,-0.270508,0.817871,0.752930,-0.822754,0.538086,1.345703,-1.120117,0.175171,-0.451416,-0.237061,-0.038208,0.803711,0.408447,-0.009430,0.798340,-0.137451,0.141235,-0.206055,0.502441,0.219482,0.215210,70.000000,0
5,2.0,-0.426025,0.960449,1.140625,-0.168213,0.420898,-0.029724,0.476318,0.260254,-0.568848,-0.371338,1.340820,0.359863,-0.358154,-0.137085,0.517578,0.401611,-0.058136,0.068665,-0.033203,0.084961,-0.208252,-0.560059,-0.026398,-0.371338,-0.232788,0.105896,0.253906,0.081055,3.669922,0
6,4.0,1.229492,0.140991,0.045380,1.202148,0.191895,0.272705,-0.005157,0.081238,0.464844,-0.099243,-1.416992,-0.153809,-0.750977,0.167358,0.050140,-0.443604,0.002821,-0.611816,-0.045563,-0.219604,-0.167725,-0.270752,-0.154053,-0.780273,0.750000,-0.257324,0.034515,0.005169,4.988281,0
7,7.0,-0.644043,1.417969,1.074219,-0.492188,0.948730,0.428223,1.121094,-3.808594,0.615234,1.249023,-0.619629,0.291504,1.757812,-1.324219,0.686035,-0.076111,-1.221680,-0.358154,0.324463,-0.156738,1.943359,-1.015625,0.057495,-0.649902,-0.415283,-0.051636,-1.207031,-1.084961,40.812500,0
8,7.0,-0.894043,0.286133,-0.113220,-0.271484,2.669922,3.722656,0.370117,0.851074,-0.392090,-0.410400,-0.705078,-0.110474,-0.286133,0.074341,-0.328857,-0.210083,-0.499756,0.118774,0.570312,0.052734,-0.073425,-0.268066,-0.204224,1.011719,0.373291,-0.384277,0.011749,0.142456,93.187500,0
9,9.0,-0.338379,1.119141,1.043945,-0.222168,0.499268,-0.246704,0.651367,0.069519,-0.736816,-0.366943,1.017578,0.836426,1.006836,-0.443604,0.150269,0.739258,-0.541016,0.476562,0.451660,0.203735,-0.246948,-0.633789,-0.120789,-0.385010,-0.069763,0.094177,0.246216,0.083069,3.679688,0


In [156]:
X = df.drop('Class', axis=1).copy()
X.head(10)

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount
0,0.0,-1.359375,-0.072754,2.537109,1.377930,-0.338379,0.462402,0.239624,0.098694,0.363770,0.090820,-0.551758,-0.617676,-0.991211,-0.311279,1.467773,-0.470459,0.208008,0.025787,0.404053,0.251465,-0.018311,0.277832,-0.110474,0.066956,0.128540,-0.189087,0.133545,-0.021057,149.625000
1,0.0,1.191406,0.266113,0.166504,0.448242,0.060028,-0.082336,-0.078796,0.085083,-0.255371,-0.166992,1.612305,1.065430,0.489014,-0.143799,0.635742,0.463867,-0.114807,-0.183350,-0.145752,-0.069092,-0.225830,-0.638672,0.101318,-0.339844,0.167114,0.125854,-0.008980,0.014725,2.689453
2,1.0,-1.358398,-1.339844,1.773438,0.379883,-0.503418,1.800781,0.791504,0.247681,-1.514648,0.207642,0.624512,0.066101,0.717285,-0.165894,2.345703,-2.890625,1.110352,-0.121338,-2.261719,0.524902,0.248047,0.771484,0.909180,-0.689453,-0.327637,-0.139038,-0.055359,-0.059753,378.750000
3,1.0,-0.966309,-0.185181,1.792969,-0.863281,-0.010307,1.247070,0.237549,0.377441,-1.386719,-0.054962,-0.226440,0.178223,0.507812,-0.287842,-0.631348,-1.059570,-0.684082,1.965820,-1.232422,-0.208008,-0.108276,0.005272,-0.190308,-1.175781,0.647461,-0.221924,0.062744,0.061462,123.500000
4,2.0,-1.158203,0.877930,1.548828,0.403076,-0.407227,0.095947,0.592773,-0.270508,0.817871,0.752930,-0.822754,0.538086,1.345703,-1.120117,0.175171,-0.451416,-0.237061,-0.038208,0.803711,0.408447,-0.009430,0.798340,-0.137451,0.141235,-0.206055,0.502441,0.219482,0.215210,70.000000
5,2.0,-0.426025,0.960449,1.140625,-0.168213,0.420898,-0.029724,0.476318,0.260254,-0.568848,-0.371338,1.340820,0.359863,-0.358154,-0.137085,0.517578,0.401611,-0.058136,0.068665,-0.033203,0.084961,-0.208252,-0.560059,-0.026398,-0.371338,-0.232788,0.105896,0.253906,0.081055,3.669922
6,4.0,1.229492,0.140991,0.045380,1.202148,0.191895,0.272705,-0.005157,0.081238,0.464844,-0.099243,-1.416992,-0.153809,-0.750977,0.167358,0.050140,-0.443604,0.002821,-0.611816,-0.045563,-0.219604,-0.167725,-0.270752,-0.154053,-0.780273,0.750000,-0.257324,0.034515,0.005169,4.988281
7,7.0,-0.644043,1.417969,1.074219,-0.492188,0.948730,0.428223,1.121094,-3.808594,0.615234,1.249023,-0.619629,0.291504,1.757812,-1.324219,0.686035,-0.076111,-1.221680,-0.358154,0.324463,-0.156738,1.943359,-1.015625,0.057495,-0.649902,-0.415283,-0.051636,-1.207031,-1.084961,40.812500
8,7.0,-0.894043,0.286133,-0.113220,-0.271484,2.669922,3.722656,0.370117,0.851074,-0.392090,-0.410400,-0.705078,-0.110474,-0.286133,0.074341,-0.328857,-0.210083,-0.499756,0.118774,0.570312,0.052734,-0.073425,-0.268066,-0.204224,1.011719,0.373291,-0.384277,0.011749,0.142456,93.187500
9,9.0,-0.338379,1.119141,1.043945,-0.222168,0.499268,-0.246704,0.651367,0.069519,-0.736816,-0.366943,1.017578,0.836426,1.006836,-0.443604,0.150269,0.739258,-0.541016,0.476562,0.451660,0.203735,-0.246948,-0.633789,-0.120789,-0.385010,-0.069763,0.094177,0.246216,0.083069,3.679688


In [157]:
y = df.Class.copy()

In [158]:
y.head(10)

0    0
1    0
2    0
3    0
4    0
5    0
6    0
7    0
8    0
9    0
Name: Class, dtype: int8

In [159]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state=RANDOM_STATE)

In [160]:
print(X_train.shape, '\n')
print(y_train.shape, '\n')
print(X_test.shape, '\n')
print(y_test.shape)

(199364, 30) 

(199364,) 

(85443, 30) 

(85443,)


In [161]:
print('--Origin dataset--','\n', df.Class.value_counts(), '\n')
print('--Train dataset--', '\n', y_train.value_counts(), '\n')
print('--Test dataset--', '\n', y_test.value_counts())

--Origin dataset-- 
 0    284315
1       492
Name: Class, dtype: int64 

--Train dataset-- 
 0    199020
1       344
Name: Class, dtype: int64 

--Test dataset-- 
 0    85295
1      148
Name: Class, dtype: int64


---Для поиска по сетке параметров задайте такие параметры:
`parameters = [{'n_estimators': [10, 15], 'max_features': np.arange(3, 5),'max_depth': np.arange(4, 7)}]`

Создайте модель `GridSearchCV` со следующими аргументами:
`estimator=RandomForestClassifier(random_state=100), param_grid=parameters, scoring='roc_auc', cv=3.`

Обучите модель на тренировочном наборе данных (может занять несколько минут).

Просмотрите параметры лучшей модели с помощью атрибута `best_params_`.

In [162]:
parameters = [{
    'n_estimators': [10, 15], 
    'max_features': np.arange(3, 5), 
    'max_depth': np.arange(4, 7)
}]

In [163]:
GS_model = GridSearchCV(
    estimator=RandomForestClassifier(random_state=100),
    param_grid=parameters,
    scoring='roc_auc',
    cv=3
)

In [164]:
GS_model.fit(X_train, y_train)

GridSearchCV(cv=3, estimator=RandomForestClassifier(random_state=100),
             param_grid=[{'max_depth': array([4, 5, 6]),
                          'max_features': array([3, 4]),
                          'n_estimators': [10, 15]}],
             scoring='roc_auc')

In [165]:
GS_model.best_params_

{'max_depth': 6, 'max_features': 4, 'n_estimators': 15}

Предскажите вероятности классов с помощью полученнной модели и метода `predict_proba`.

Из полученного результата (массив `Numpy`) выберите столбец с индексом 1 (вероятность класса 1) и запишите в массив **y_pred_proba**. Из модуля `sklearn.metrics` импортируйте метрику `roc_auc_score`.
Вычислите `AUC` на тестовых данных и сравните с результатом, полученным на тренировочных данных, используя в качестве аргументов массивы **y_test** и **y_pred_proba**.

In [166]:
y_pred_proba = GS_model.predict_proba(X_test)[:, 1]
y_pred_proba

array([0.00088048, 0.0003038 , 0.00022513, ..., 0.00019673, 0.00021123,
       0.00019673])

In [167]:
roc_auc_score(y_true=y_test, y_score=y_pred_proba)

0.957606035016786